<a href="https://colab.research.google.com/github/CarlosRochaCx/MNA/blob/main/MNA_IAyAA_Actividad_semanas_6y7_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Inteligencia Artificial y Aprendizaje Automático**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad de Semanas 6 y 7

### **Campañas publicitarias en redes sociales - Modelos de Regresión**



### **Nombres y Matrículas:**


*   Elemento de lista
*   Elemento de lista
*   Elemento de lista


### **Trabajaremos con el archivo "dataset_Facebook.csv" que encuentras en la siguiente liga:**

https://archive.ics.uci.edu/dataset/368/facebook+metrics

### **Estos datos están asociados al siguiente artículo de Moro et.al. de ELSEVIER, que deberás descargar para contestar varias de las preguntas de esta actividad (el acceso es sin costo alguno):**

https://www.semanticscholar.org/paper/Predicting-social-media-performance-metrics-and-of-Moro-Rita/dec55692590820754b53c916e29bb2b42c0e5104


## **NOTA: No modifiques el código, salvo en las partes que se te indica.**

In [1]:
# Puedes incluir más librerías, de ser necesario:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error, r2_score

np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')

## **Ejercicio 1**


### **De acuerdo al artículo de Moro et.al. de ELSEVIER, contesta las siguientes preguntas:**

*   **a. ¿Cuál es el objetivo del problema que se plantea en el artículo?**


### ++++++++ Inicia la sección de agregar texto: +++++++++++

El objetivo principal del artículo es crear un modelo que pueda predecir cómo van a funcionar las publicaciones en la página de Facebook de una marca antes de que se publiquen. Para hacerlo, se usaron técnicas de minería de datos y se analizaron siete características de cada publicación, como el tipo de contenido, el día de la semana y si la publicación fue pagada o no. Con esta información, el modelo intenta anticipar el nivel de interacción que tendrá cada publicación, como la cantidad de personas que la verán o interactuarán con ella.

Además de hacer estas predicciones, el estudio busca ayudar a los encargados de marketing a tomar mejores decisiones sobre qué tipo de contenido publicar para tener mayor impacto en redes sociales. También se analiza cómo estas decisiones pueden ayudar a fortalecer la imagen de la marca, ya que una buena estrategia de publicaciones puede aumentar la confianza y el reconocimiento del público hacia la empresa.

### ++++++++ Termina la sección de agregar texto. +++++++++++


*   **b. Describe a continuación el significado de cada una de las 8 variables con la cuales trabajaremos, de acuerdo a la información de las Tablas 2 y 3 del artículo de Moro et.al.**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

*   **Category**: Caracterización manual del contenido: Acción (ofertas especiales o concursos), producto (publicidad directa, contenido explícitamente de una marca) o inspiración (contenido no relacionado a alguna marca).


*   **Page total likes**: Número de personas que le dieron "me gusta" a la página de la compañía.


*   **Type**: Tipo de contenido en la publicación. (Un enlace, fotografía, estatus o video)


*   **Post month**: Indica el mes en el que se hizo la publicación.


*   **Post hour**: Indica la hora en el que se hizo la publicación.


*   **Post weekday**: Indica el día de la semana en el que se hizo la publicación.


*   **Paid**: Indica si la compaña pagó a Facebook por publicidad.


*   **Lifetime post consumers**: Número de personas que le dieron clic a cualquier parte de la publicación.



### ++++++++ Termina la sección de agregar texto. +++++++++++

### **En esta actividad trabajarás solamente con el ajuste de modelos, por lo que la parte de procesamiento no la debes modificar. Solo haremos unos pequeños ajustes.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cargamos los datos del archivo:

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MNA - Maestría en Inteligencia Artificial Aplicada/Inteligencia Artificial y Aprendizaje Automático/Tareas/Actividad Semana 6 y 7/dataset_Facebook.csv', sep=';', header='infer')
print('Total de registros y variables:',data.shape)
data.head(3).T

Total de registros y variables: (500, 19)


,0,1,2
Page total likes,139441,139441,139441
Type,Photo,Status,Photo
Category,2,2,3
Post Month,12,12,12
Post Weekday,4,3,3
Post Hour,3,10,3
Paid,0.0,0.0,0.0
Lifetime Post Total Reach,2752,10460,2413
Lifetime Post Total Impressions,5091,19057,4373
Lifetime Engaged Users,178,1457,177


## **Ejercicio 2**

### **Variables relacionadas con el tiempo**

### **Recordemos que las variables relacionadas con información del tiempo (meses, semanas, horas) pueden tratarse como variables categóricas o numéricas, dependiendo del contexto. En nuestro caso las variables mes, día de la semana y hora en que se puso el post, podríamos considerarlas como variables categóricas ordinales o bien como nominales. Sin embargo, para recordar cómo se transforman este tipo de variables en numéricas cíclicas, las transformarás a continuación.**

### **En la Tabla 3 del artículo de ELSEVIER nos comentan que la variable Post_hour está etiquetada de 0 a 23, por lo que las 24 horas serían las 0 horas, es decir tiene un comportamiento cíclico de 24 horas, que en términos matemáticos se dice que es una variable cíclica módulo 24. Así, las 24 horas será equivalente a las 0 horas. De manera análoga se procede con las variables del mes y el día de la semana.**

### **Se comenta en la Tabla 3 del artículo que las variables en relación al mes y día de la semana en que se publicó el post, que son los nombres de los meses y los días de la semana. Sin embargo, en realidad están capturadas como numéricas del 1 al 12 y del 1 al 7, respectivamente.**

### **La manera de transformar una variable T cíclica módulo M, es sustituyendo la columna original de la variable T por las siguientes dos columnas. Estas transformaciones están dentro de lo que se llama ingeniería de características (feature engineering):**

$Tsin = sin(2*\pi*T/M)$

$Tcos = cos(2*\pi*T/M)$

#### **NOTA: En las siguientes celdas donde se indica None, podrás agregar las líneas de código que consideres necesarias.**

In [ ]:
# Ejercicio 2a:

# Define las dos nuevas variables cíclicas, "hora_sin" y "hora_cos" a partir
# de la variable "Post Hour" y de acuerdo a como se indicó previamente:


# ++++++++++++++ Incluye aquí tu código ++++++++++++++++++++++++++++++

hora_sin = None
hora_cos = None



# +++++++++++++ Tesrmina sección de agregar código +++++++++++++++++++++

In [ ]:
# Ejercicio 2b:

# Agrega estas dos nuevas variables al DataFrame de tus datos
# y elimina la columna de la variable original "Post Hour":

# ++++++++++++++ Incluye aquí tu código ++++++++++++++++++++++++++++++

None



# +++++++++++++ Termina sección de agregar código +++++++++++++++++++++


In [ ]:
# Ejercicio 2c.

# Transforma de manera análoga la variable "Post Month"
# agregando mes_sin, mes_cos y eliminando la original.


# ++++++++++++++ Incluye aquí tu código ++++++++++++++++++++++++++++++

mes_sin = None
mes_cos = None



# +++++++++++++ Tesrmina sección de agregar código +++++++++++++++++++++


In [ ]:
# Ejercicio 2d.

# Transforma de manera análoga la variable "Post Weekday"
# agregando dia_sin, dia_cos y eliminando la original.


# ++++++++++++++ Incluye aquí tu código ++++++++++++++++++++++++++++++

dia_sin = None
dia_cos = None



# +++++++++++++ Tesrmina sección de agregar código +++++++++++++++++++++


In [ ]:
# Veamos lo que tenemos hasta el momento:
data.head(3).T

In [ ]:
# De las Tablas 2 y 3 del artículo de ELSEVIER seleccionamos las variables que
# trabajaremos en esta Actividad, con el ajuste que acabamos de hacer:

# Separamos los datos de entrada (ver Tabla 3 del artículo) y de la variable de salida (ver Tabla 2):
X = data[['Page total likes', 'Type', 'Category', 'Paid',
          'mes_sin', 'mes_cos', 'dia_sin', 'dia_cos', 'hora_sin', 'hora_cos']]
y = data[['Lifetime Post Consumers']]   # Hay 12 variables de salida, pero solo trabajaremos con una de las
                                        # que se consideró más importante en el artículo de ELSEVIER.


# En esta acrividad trataremos de comparar resultados con lo realizado en
# el artículo de Moro et.al., por lo que solo particionaremos en Train y Test.
# Particionamos en Train y Test en 80-20:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=5)

print('Train:', Xtrain.shape, ytrain.shape)
print('Test:', Xtest.shape, ytest.shape)


In [ ]:
Xtrain.describe(include='all').T   # Veamos alguna descripción como datos numéricos del conjunto de entrenamiento.
                                   # En particular, las desviaciones estándar (std) desplegadas son las muestrales.

### **Ajuste variable categórica en Pipeline**

#### **Veamos un ejemplo de cómo ajustar una variable categórica en la cual uno de sus niveles no tiene suficiente información y que se transforme evitando el filtrado de información. Para ello se deberán reagruparemos los niveles más pequeños hasta obtener un 5% de información.**

In [ ]:
# Obtengamos las etiquetas únicas de data['Type'] ordenadas de mayor a menor frecuencia:
Xtrain['Type'].unique()

In [ ]:
Xtrain['Type'].value_counts(normalize=True)  # Siguiendo la política de que cada nivel de una variable tenga
                                             # al menos el 5% de información, agruparemos los casos "Link"
                                             # y "Video" en un nuevo nivel que podría interpretarse como "Otros".

### **Para ello usaremos la siguiente Clase, que ayude a cuidar el filtrado de información durante el proceso de entrenamiento, en particular, al aplicar validación cruzada.**

In [ ]:
# Transformador personalizado para agrupar categorías poco frecuentes
# en la variable "Type", a saber, "Link" y "Video".
# Puedes posteriormente ajustar la clase para que de manera automática
# se detecte que cada nivel tiene al menos un porcentaje de umbral determinado.

class TypeGrouper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.groups_to_replace = ['Link','Video']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        if not isinstance(X_copy, pd.DataFrame):
            X_copy = pd.DataFrame(X_copy, columns=['Type'])

        X_copy.loc[X_copy['Type'].isin(self.groups_to_replace), 'Type'] = 'Otros'
        return X_copy


## **Ejercicio 3**

### **Utiliza un LLM (indica cuál o cuáles usastes) para explicar cada línea de código de la celda anterior.**

### ++++++++ Inicia la sección de agregar texto: +++++++++++


*   Elemento de lista
*   Elemento de lista

etc ...


### ++++++++ Termina la sección de agregar texto. +++++++++++

### **Transformación en la Variable de Salida**



In [ ]:
# Histograma que muestra un claro sesgo positivo en la variable objetivo:
ytrain.hist();

In [ ]:
ytrainlog = np.log(ytrain)   # Recordemos que nuestra variable de salida en un problema
ytrainlog.hist();            # de Regresión se recomienda que esté aproximadamente
                             # acampanada, por lo que la ajustaremos con el logaritmo
                             # natural como primera aproximación.

In [ ]:
# Para una primera aproximación, podemos decir que se ajusta de manera aceptable
# la distribución con logaritmo a una acampanada.
# Procedemos entonces de la misma manera con Test:

ytestlog = np.log(ytest)

In [ ]:
ytestlog.describe().T  # información de la variable objetivo transformada.

### **Pipeline de transformaciones**

In [ ]:
# Aplicaremos lo mínimo de transformaciones.
# Las nuevas variables _sin y _cos ya quedaron escaladas entre -1 y +1, por
# lo que podemos dejarlas así y solamente cuidar sus posibles datos perdidos
# en datos futuros.

num_pipe = Pipeline(steps = [('numImp', SimpleImputer(strategy='median')),
                             ('scaler', MinMaxScaler())
                             ])
num_pipe_nombres = ['Page total likes']


# Transformación del factor categórico de entrada "Type":
catMap_pipe = Pipeline(steps = [('typeMap', Pipeline([('grouper', TypeGrouper()),
                                                      ('encoder', OneHotEncoder(handle_unknown='ignore'))
                                                      ]))])
catMap_pipe_nombres = ['Type']


# Las nominales las transformamos con One-Hot-Encoder:
nom_pipe = Pipeline(steps = [('NomImp', SimpleImputer(strategy='most_frequent')),
                             ('ohe', OneHotEncoder(drop='first',
                                                   handle_unknown='ignore',
                                                   ))])
nom_pipe_nombres = ['Category', 'Paid']


# Variables numéricas cíclicas:
mes_pipe = Pipeline(steps = [('MesImp', SimpleImputer(strategy='mean'))])
mes_pipe_nombres = ['mes_sin', 'mes_cos']

dia_pipe = Pipeline(steps = [('DiaImp', SimpleImputer(strategy='mean'))])
dia_pipe_nombres = ['dia_sin', 'dia_cos']

hora_pipe = Pipeline(steps = [('HoraImp', SimpleImputer(strategy='mean'))])
hora_pipe_nombres = ['hora_sin', 'hora_cos']


# Conjuntamos las transformaciones:
columnasTransformer = ColumnTransformer(transformers = [('numpow', num_pipe, num_pipe_nombres),
                                                        ('catmap', catMap_pipe, catMap_pipe_nombres),
                                                        ('catNom', nom_pipe, nom_pipe_nombres),
                                                        ('mes', mes_pipe, mes_pipe_nombres),
                                                        ('dia', dia_pipe, dia_pipe_nombres),
                                                        ('hora', hora_pipe, hora_pipe_nombres)
                                                        ],
                                        remainder='passthrough')


In [ ]:
# Solo para saber la nueva cantidad de columnas que tendremos
# después de las trasnformaciones:
XtrainT = columnasTransformer.fit_transform(Xtrain)  # Ajustamos con Train...
print('Variables de entrada original:', Xtrain.shape)
print('Variables de entrada transformadas:', XtrainT.shape)

### **Modelo Base (Baseline)**

In [ ]:
# Variable objetivo con Log:
y_pred_baseline = np.full_like(ytestlog, fill_value=ytrainlog.mean())

# RMSE del baseline
rmse_baseline = mean_squared_error(ytestlog, y_pred_baseline)
print(f"MSE Baseline (promedio) con Logaritmo: {rmse_baseline:.2f}")

## **Ejercicio 4**


In [ ]:
# Ejercicio 4a.

# Al ejecutar las siguientes líneas de código determina si el modelo de
# Bosque Aleatorio con sus valores de hiperparámetros predeterminadas está
# Subentrenado o Sobreentrenado. De ser así, busca los valores de sus
# hiperparámetros que consideres más adecuados para que ya no
# esté sub-o-sobreentrenado:


# ++++++++++++++ Incluye aquí tu código ++++++++++++++++++++++++++++++


elmodelo_RF = RandomForestRegressor(random_state=1)





# +++++++++++++ Tesrmina sección de agregar código +++++++++++++++++++++



mipipe = Pipeline(steps=[('ct',columnasTransformer),('m', elmodelo_RF)])

train_sizes, train_scores, val_scores = learning_curve(estimator=mipipe,
                                                        X=Xtrain,
                                                        y=np.ravel(ytrainlog),
                                                        cv=5,
                                                        train_sizes= np.linspace(0.1, 1.0, 10),
                                                        scoring= 'neg_mean_squared_error',  # error MSE
                                                        n_jobs=-1)



# Calculamos los promedios y desviación estándar de entrenamiento para MSE.
# El negativo es porque sklearn nos devuelve en realidad el negativo del error
# cuadrático medio MSE.

mse_train_scores = -train_scores
mse_val_scores = -val_scores

train_mean = np.mean(mse_train_scores, axis=1)
train_mse = np.std(mse_train_scores, axis=1, ddof=1)
val_mean = np.mean(mse_val_scores, axis=1)
val_mse = np.std(mse_val_scores, axis=1, ddof=1)


plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training MSE')
plt.fill_between(train_sizes, train_mean + train_mse, train_mean - train_mse, alpha=0.15, color='blue')

plt.plot(train_sizes, val_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation MSE')
plt.fill_between(train_sizes, val_mean + val_mse, val_mean - val_mse, alpha=0.15, color='green')

plt.title('Curvas de Aprendizaje (Learning Curves)')
plt.xlabel('Tamaño de la muestra de entrenamiento')
plt.ylabel('Error MSE')
plt.grid()
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Ejercicio 4b.

# Una vez que verifiques que no está sobreentrenado o subentrenado el modelo,
# podemos calcular el MSE del mejor ajuste con los datos de Prueba.
# Incluye los valores de tus mejores hiperparámetros encontrados del RandomForest.

# ++++++++++++++ Incluye aquí tus ajustes ++++++++++++++++++++++++++++++


mejor_modelo_RF = RandomForestRegressor(random_state=1)





# +++++++++++++ Tesrmina sección de realizar ajustes +++++++++++++++++++++


mipipe = Pipeline(steps=[('ct',columnasTransformer),('m', mejor_modelo_RF)])
mipipe.fit(pd.DataFrame(Xtrain, columns=Xtrain.columns), np.ravel(ytrainlog))
yhattest_RF = mipipe.predict(Xtest)

print('Error MSE(Test) de Random Forest: %.3f' % mean_squared_error(np.ravel(ytestlog), yhattest_RF))

## **Ejercicio 5**

In [ ]:
# Ejercicio 5:
# Calcula ahora el error porcencual absoluto medio (MAPE por sus siglas en inglés)
# del mejor modelo de Random Forest en el conjunto de Prueba (TEST).
# Llamarle "mape" a dicha variable.


# ++++++++++++++ Incluye aquí tus ajustes ++++++++++++++++++++++++++++++


None



# +++++++++++++ Tesrmina sección de realizar ajustes +++++++++++++++++++++


# Imprimimos el resultado de MAPE en porcentaje:
print(f"MAPE: {mape:.2f}%")


## **Ejercicio 6**

### **Repite el mismo proceso del ejercicio con el modelo Bosque Aleatorio (Random Forest) para obtener los mejores modelos no subentrenados y no sobreentrenados, en cada uno de los siguientes casos:**

*   **a. Extreme Gradient Boost XGBoost**
*   **b. Máquina de Vector Soporte SVM**
*   **c. Red Neuronal Perceptrón Multicapa MLP**

In [ ]:
# Incluye a continuación todas las celdas y líneas de código que consideres
# necesarias para responder el Ejercicio 6a, 6b y 6c.




None




## **Ejercicio 7**

### **Incluye tus conclusiones finales de la Actividad, en particular compara el resultado MAPE de tu mejor modelo del conjunto de Prueba (Test) con el resultado reportado en el artículo de Moro et.al en la sección de conclusiones.**

### ++++++++ Inicia la sección de agregar texto: +++++++++++


None


### ++++++++ Termina la sección de agregar texto. +++++++++++

## **Fin de la Actividad de la Semanas 6 y 7**